In [3]:
import requests
from io import BytesIO
import gzip
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from shapely.geometry import mapping
from rasterio.features import geometry_mask, shapes
from scipy.interpolate import NearestNDInterpolator
import matplotlib.pyplot as plt
import startinpy
from rasterio import Affine
from shapely.geometry import shape,box
from rasterio.crs import CRS
from pathlib import Path
import laspy
from scipy.spatial import cKDTree
from scipy.ndimage import median_filter, label

In [4]:
input_pc = "output/pointcloud.las"


In [5]:

def write_output(dataset, crs, output, transform, name, change_nodata=False):
    """
    Write grid to .tiff file.
    ----
    Input:
    - dataset: Can be either a rasterio dataset (for rasters) or laspy dataset (for point clouds)
    - output (Array): the output grid, a numpy grid.
    - name (String): the name of the output file.
    - transform:
      a user defined rasterio Affine object, used for the transforming the pixel coordinates
      to spatial coordinates.
    - change_nodata (Boolean): true: use a no data value of -9999, false: use the datasets no data value
    """
    output_file = name

    output = np.squeeze(output)

    # Set the nodata value: use -9999 if nodata_value is True or dataset does not have nodata.
    if change_nodata:
        nodata_value = -9999
    else:
        try:
            nodata_value = dataset.nodata
            if nodata_value is None:
                raise AttributeError("No no data value found in dataset.")
        except AttributeError as e:
            print(f"Warning: {e}. Defaulting to -9999.")
            nodata_value = -9999


    # output the dataset
    with rasterio.open(output_file, 'w',
                       driver='GTiff',
                       height=output.shape[0],  # Assuming output is (rows, cols)
                       width=output.shape[1],
                       count=1,
                       dtype=np.float32,
                       crs=crs,
                       nodata=nodata_value,
                       transform=transform) as dst:
        dst.write(output, 1)
    print("File written to '%s'" % output_file)

def raster_center_coords(min_x, max_x, min_y, max_y, resolution):
    """
    Compute the center xy coordinates of a grid.
    ----
    Input:
    - min_x, max_x, min_y, max_y(float): Minimum and maximum x and y coordinates of the grid.
    - resolution (float): The length of each cell, function can only be used for square cells.

    Output:
    - grid_center_x: a grid where each cell contains the value of its center point's x coordinates.
    - grid_center_y: a grid where each cell contains the value of its center point's y coordinates.
    """
    # create coordinates for the x and y border of every cell.
    x_coords = np.arange(min_x, max_x, resolution)  # x coordinates expand from left to right.
    y_coords = np.arange(max_y, min_y, -resolution)  # y coordinates reduce from top to bottom.

    # create center point coordinates for evey cell.
    grid_x, grid_y = np.meshgrid(x_coords, y_coords)
    grid_center_x = grid_x + resolution / 2
    grid_center_y = grid_y - resolution / 2
    return grid_center_x, grid_center_y

def dem_creation(LasData, data, output_filename, resolution=0.5, smooth=False, nodata_value=-9999, filter_size=3):
    raster = data[0]
    grid_centers = data[1]
    top_left_x = grid_centers[0][0, 0] - resolution / 2
    top_left_y = grid_centers[1][0, 0] + resolution / 2

    transform =Affine.translation(top_left_x, top_left_y) * Affine.scale(resolution, -resolution)
    crs = CRS.from_epsg(28992)

    write_output(LasData, crs, raster, transform, output_filename, True)


def interpolation(LasData, points, resolution, no_data_value=-9999):
    """
    Create a vegetation raster using Laplace interpolation.

    InpurL
    - LasData (laspy.LasData):          Input LiDAR point cloud data.
    - veg_points (laspy.LasData):       Vegetation points to be interpolated.
    - resolution (float):               Resolution of the raster.
    - no_data_value (int, optional):    Value for no data

    Returns:
    - interpolated_grid (np.ndarray): Generated raster for vegetation.
    - grid_center_xy (tuple): Grid of x, y center coordinates for each raster cell.
    """

    # Extents of the pc
    min_x, max_x = round(LasData.x.min()), round(LasData.x.max())
    min_y, max_y = round(LasData.y.min()), round(LasData.y.max())

    # Define size of the region
    x_length = max_x - min_x
    y_length = max_y - min_y

    # Number of rows and columns
    cols = round(x_length / resolution)
    rows = round(y_length / resolution)

    # Initialize raster grid
    vege_raster = np.full((rows, cols), no_data_value, dtype=np.float32)

    # Calculate center coords for each grid cell
    grid_center_xy = raster_center_coords(min_x, max_x, min_y, max_y, resolution)

    if points.x.shape[0] == 0:
        print("There are no vegetation points in the current area.")
        vege_raster = np.full((rows, cols), -200, dtype=np.float32)
        return vege_raster, grid_center_xy

    # create the delaunay triangulation
    dt = startinpy.DT()
    dt.insert(points.xyz, "BBox")

    # Flatten the grid to get a list of all center coords
    locs = np.column_stack((grid_center_xy[0].ravel(), grid_center_xy[1].ravel()))


    # laplace interpolation
    interpolated_values = dt.interpolate({"method": "Laplace"}, locs)

    # reshape interpolated grid back to og
    interpolated_grid = np.full_like(vege_raster, no_data_value, dtype=np.float32)  # Start with no_data
    interpolated_grid.ravel()[:] = interpolated_values

    return interpolated_grid, grid_center_xy

In [6]:
with laspy.open(input_pc, laz_backend=laspy.LazBackend.Lazrs) as las:
    las_data = las.read()
#
# dst = rasterio.open("D:\Geomatics\thesis\wcs_test\maps\final_dtm_wcs.tif", "r")

In [7]:
building_ground_points = las_data[(las_data.classification == 2) | (las_data.classification == 6)]
ground_points =  building_ground_points[(building_ground_points.classification == 2)]
all_data = interpolation(las_data, building_ground_points, 0.5)
dem_creation(las_data, all_data, "D:/Geomatics/thesis/wcs_test/maps/final_dsm_pc.tif")
ground_pointsoutput = interpolation(las_data, ground_points, 0.5)
dem_creation(las_data, ground_pointsoutput, "D:/Geomatics/thesis/wcs_test/maps/final_dtm_pc.tif")

File written to 'D:/Geomatics/thesis/wcs_test/maps/final_dsm_pc.tif'
File written to 'D:/Geomatics/thesis/wcs_test/maps/final_dtm_pc.tif'
